In [1]:
# Basic imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import scipy.optimize as spo
import sys
from time import time
from sklearn.metrics import r2_score, median_absolute_error

%matplotlib inline

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

sys.path.append('../../')

from utils import preprocessing as pp

Populating the interactive namespace from numpy and matplotlib


In [2]:
%pwd

'/home/miguel/udacity/Machine Learning Nanodegree/projects/capstone/capstone/notebooks/dev'

In [3]:
# Getting the data
data_df = pd.read_pickle('../../data/data_train_val_df.pkl')

sys.path.append('../../')
import predictor.feature_extraction as fe

train_time = -1 # In real time days
base_days = 7 # In market days
step_days = 7 # market days
ahead_days = 1 # market days
today = data_df.index[-1] # Real date

tic = time()
x, y = fe.generate_train_intervals(data_df, 
                                   train_time, 
                                   base_days, 
                                   step_days, 
                                   ahead_days, 
                                   today, 
                                   fe.feature_close_one_to_one)
toc = time()
print('Elapsed time: %i seconds.' % (toc-tic))

Elapsed time: 183 seconds.


In [4]:
print(x.shape)
x.head()

(314954, 7)


,,0,1,2,3,4,5,6
2003-10-20,YHOO,21.18,21.41,20.88,20.20,20.26,20.58,21.51
2007-04-03,BEN,236.54,235.75,235.99,234.34,233.48,232.50,233.93
1996-11-22,KMI,9.91,10.03,10.03,10.06,10.00,10.47,10.31
2012-07-16,XRAY,50.32,51.29,51.48,52.09,51.10,49.90,49.20
2010-09-14,VRSN,37.87,37.82,37.16,36.63,36.63,37.78,36.80


In [5]:
y.shape

(314954, 1)

In [6]:
x_y = pd.concat([x, y], axis=1)
x_y.shape

(314954, 8)

In [7]:
x_y_sorted = x_y.sort_index()
x_y_sorted.head()

0      1      2      3      4      5      6  target
1993-01-29 AAL   18.75  19.12  20.25  20.50  20.12  19.62  19.75   19.75
           ABBV   2.12   2.19   2.15   2.14   2.12   2.04   2.02    2.03
           ABC    2.62   2.62   2.56   2.62   2.69   2.62   2.44    2.44
           ADI   21.94  22.38  22.31  22.69  23.19  24.00  23.25   22.75
           ADP   15.25  15.25  15.06  15.56  15.69  15.81  15.75   15.69

In [8]:
x_y_sorted

0       1       2       3       4       5       6  \
1993-01-29 AAL    18.75   19.12   20.25   20.50   20.12   19.62   19.75   
           ABBV    2.12    2.19    2.15    2.14    2.12    2.04    2.02   
           ABC     2.62    2.62    2.56    2.62    2.69    2.62    2.44   
           ADI    21.94   22.38   22.31   22.69   23.19   24.00   23.25   
           ADP    15.25   15.25   15.06   15.56   15.69   15.81   15.75   
           ADS     7.00    7.06    7.25    7.25    8.00    8.12    8.19   
           ADSK   13.44   13.50   13.66   13.72   14.00   13.75   13.59   
           AEE     1.09    1.13    1.19    1.20    1.21    1.17    1.17   
           AEP     2.41    2.41    2.41    2.41    2.41    2.39    2.37   
           AES    11.24   11.03   11.35   11.24   11.24   11.24   11.08   
           AET    11.53   11.79   11.87   12.21   12.35   12.46   12.49   
           AFL     5.94    5.97    5.89    5.94    5.80    5.45    5.66   
           AIG     7.75    7.50    7.38    7.16    7.47    6.73    6.22   
           AIZ    17.56   17.12   17.19   17.88   18.12   18.06   17.62   
           AKAM    4.41    4.53    4.53    4.49    4.49    4.37    4.45   
           ALB     4.49    4.52    4.57    4.65    4.84    4.80    4.82   
           ALK     6.42    6.64    6.62    6.50    6.73    6.66    6.67   
           ALXN  250.30  252.42  257.96  265.88  268.26  264.83  261.92   
           AMAT   13.44   13.56   13.38   13.50   13.69   13.94   13.75   
           AMD     2.59    2.72    2.84    2.70    2.73    2.60    2.62   
           AN      2.05    2.12    2.14    2.12    2.11    2.08    2.08   
           ANTM    1.94    2.00    2.03    2.09    2.09    1.97    1.97   
           APA   223.13  219.38  221.25  222.00  228.00  232.88  229.50   
           APC     7.50    7.47    7.53    7.56    7.53    7.75    8.00   
           APD    33.88   34.12   34.25   34.88   34.50   34.50   34.62   
           APH     9.25    9.36    9.36    9.41    9.36    9.52    9.46   
           ARNC    6.77    7.18    7.18    7.25    7.25    7.28    7.36   
           AVB     1.50    1.53    1.51    1.53    1.70    1.72    1.70   
           AVY    16.69   16.62   16.62   16.94   17.38   17.19   16.94   
           AYI     6.88    6.88    6.53    6.91    6.84    6.88    6.66   
...                 ...     ...     ...     ...     ...     ...     ...   
2014-12-16 VRSN   62.12   63.99   65.43   66.78   66.09   66.29   66.27   
           VRTX   55.54   56.26   57.30   58.29   58.49   58.46   58.28   
           VTR    47.03   47.79   47.07   48.88   48.84   49.36   49.79   
           VZ     62.77   63.75   64.85   64.70   65.15   64.92   64.88   
           WAT    47.88   48.56   48.96   48.80   49.07   49.07   48.67   
           WBA   104.49  107.95  112.36  113.88  114.28  113.30  113.11   
           WDC    41.84   43.63   44.80   46.00   45.64   45.73   45.35   
           WEC   174.43  188.93  190.52  189.73  190.29  191.13  191.15   
           WFC   114.99  118.22  118.38  119.20  119.28  120.05  119.95   
           WFM    81.48   82.90   84.31   84.76   84.71   85.16   85.68   
           WHR    34.74   35.01   35.30   36.19   36.49   36.58   36.62   
           WLTW  100.94  106.82  108.67  113.62  111.77  113.62  111.95   
           WM     52.77   53.79   55.21   54.45   54.97   55.47   55.34   
           WMB   140.52  138.97  145.10  150.11  147.48  151.69  150.90   
           WU     82.96   84.23   85.94   85.16   86.38   86.66   86.43   
           WY     16.95   17.47   17.78   18.00   18.05   18.12   18.07   
           WYN    34.22   34.75   35.35   35.40   35.65   35.43   36.23   
           WYNN   43.70   43.42   43.71   43.00   43.87   43.91   44.08   
           XEC    34.97   35.75   36.43   36.49   37.00   37.20   36.97   
           XEL    86.41   89.02   91.16   93.64   93.33   94.59   93.78   
           XLNX   13.35   13.46   13.89   13.96   13.97   14.07   14.08   
           XOM    19.31   19.60   20.09   20.07

In [9]:
x_y_sorted.loc[x_y_sorted.index.levels[0][-1]:]

0       1       2       3       4       5       6  \
2014-12-16 A      42.67   44.06   45.01   45.12   45.41   45.63   46.84   
           AAL     2.50    2.55    2.55    2.57    2.66    2.67    2.65   
           AAP    54.78   55.99   57.06   56.75   57.49   56.66   56.71   
           AAPL  161.09  164.90  165.89  165.60  169.20  168.22  166.95   
           ABBV  106.74  109.41  112.65  111.78  112.94  112.54  112.01   
           ABC    55.92   57.07   57.75   58.56   58.63   59.41   59.74   
           ABT   157.01  159.55  160.74  159.85  159.44  161.19  160.23   
           ACN    65.25   66.57   67.92   67.71   66.97   64.35   66.21   
           ADBE   82.87   85.30   89.74   90.51   91.18   91.42   91.32   
           ADI   136.82  140.96  144.45  147.04  147.83  148.58  148.58   
           ADM    51.42   51.76   52.95   52.36   52.49   53.17   53.76   
           ADP    70.73   72.27   74.07   74.48   74.70   74.49   74.45   
           ADS    46.28   46.91   47.43   47.42   47.59   47.48   47.45   
           ADSK   16.72   17.26   17.53   17.62   17.71   17.93   17.98   
           AEE    23.85   24.46   25.01   24.89   25.40   25.40   25.25   
           AEP    48.73   50.11   52.29   52.36   53.14   53.79   53.98   
           AES    49.32   49.64   51.16   52.24   52.72   53.17   53.11   
           AET    82.91   84.68   85.68   84.71   85.70   85.17   85.24   
           AFL    56.63   57.01   59.46   59.90   60.19   60.65   60.99   
           AGN    65.07   66.19   67.71   68.65   68.71   69.23   69.17   
           AIG   159.44  163.48  168.48  170.17  164.53  157.52  160.67   
           AIV    19.38   19.67   20.04   20.25   20.30   20.23   20.35   
           AIZ   597.93  602.46  615.48  618.60  616.64  621.30  617.43   
           AJG   128.36  132.98  137.17  137.38  140.05  139.48  139.94   
           AKAM   12.79   13.13   13.21   13.50   13.66   13.65   13.81   
           ALB    57.81   59.07   60.32   61.18   61.66   61.94   61.94   
           ALK    85.64   87.18   90.13   90.84   90.41   89.45   89.59   
           ALL    54.97   57.20   58.84   58.69   60.00   60.82   60.48   
           ALLE   35.66   36.12   36.80   37.51   38.32   38.13   37.80   
           ALXN   52.88   54.02   55.01   55.78   56.13   56.44   56.28   
...                 ...     ...     ...     ...     ...     ...     ...   
           VRSN   62.12   63.99   65.43   66.78   66.09   66.29   66.27   
           VRTX   55.54   56.26   57.30   58.29   58.49   58.46   58.28   
           VTR    47.03   47.79   47.07   48.88   48.84   49.36   49.79   
           VZ     62.77   63.75   64.85   64.70   65.15   64.92   64.88   
           WAT    47.88   48.56   48.96   48.80   49.07   49.07   48.67   
           WBA   104.49  107.95  112.36  113.88  114.28  113.30  113.11   
           WDC    41.84   43.63   44.80   46.00   45.64   45.73   45.35   
           WEC   174.43  188.93  190.52  189.73  190.29  191.13  191.15   
           WFC   114.99  118.22  118.38  119.20  119.28  120.05  119.95   
           WFM    81.48   82.90   84.31   84.76   84.71   85.16   85.68   
           WHR    34.74   35.01   35.30   36.19   36.49   36.58   36.62   
           WLTW  100.94  106.82  108.67  113.62  111.77  113.62  111.95   
           WM     52.77   53.79   55.21   54.45   54.97   55.47   55.34   
           WMB   140.52  138.97  145.10  150.11  147.48  151.69  150.90   
           WU     82.96   84.23   85.94   85.16   86.38   86.66   86.43   
           WY     16.95   17.47   17.78   18.00   18.05   18.12   18.07   
           WYN    34.22   34.75   35.35   35.40   35.65   35.43   36.23   
           WYNN   43.70   43.42   43.71   43.00   43.87   43.91   44.08   
           XEC    34.97   35.75   36.43   36.49   37.00   37.20   36.97   
           XEL    86.41   89.02   91.16   93.64   93.33   94.59   93.78   
           XLNX   13.35   13.46   13.89   13.96   13.97   14.07   14.08   
           XOM    19.31   19.60   20.09   20.07

In [10]:
x_y_sorted.index.levels[0][-1]

Timestamp('2014-12-16 00:00:00')

In [11]:
fe.add_market_days(x_y_sorted.index.levels[0][-1], -ahead_days)

Timestamp('2014-12-15 00:00:00')

In [12]:
x_train = x_y_sorted.loc[:fe.add_market_days(x_y_sorted.index.levels[0][-1], -ahead_days)]

In [13]:
x_train.shape

(314475, 8)

In [14]:
x_y_val = x_y_sorted.loc[x_y_sorted.index[-1]]
pd.DataFrame(x_y_val).set_index(x_y_val.name, append=True)

,"(2014-12-16 00:00:00, ZTS)"
0,40.85
1,41.13
2,43.15
3,43.51
4,43.41
5,42.97
6,43.84
target,44.20


In [15]:
def reshape_by_symbol(y):
    # TODO: Fix multi/single level
    
    multiindex = y.index.nlevels > 1
    
    if multiindex:
        DATE_LEVEL_NAME = 'level_0'
    else:
        DATE_LEVEL_NAME = 'index'
    grouped_df = y.reset_index() \
        .groupby('level_0') \
        .apply(lambda x: x.reset_index(drop=True)) \
        .drop('level_0', axis=1)
    grouped_df.index = grouped_df.index.droplevel(level=1)
    grouped_df.rename(columns={'level_1':'ticker'}, inplace=True)
    reshaped_df = grouped_df.set_index('ticker', append=True).unstack()
    reshaped_df.columns = reshaped_df.columns.droplevel(level=0)
    reshaped_df.index.name = 'date'
    return reshaped_df

In [16]:
def run_single_val(x, y, ahead_days, estimator):
    x_y = pd.concat([x, y], axis=1)
    x_y_sorted = x_y.sort_index()
    x_y_train = x_y_sorted.loc[:fe.add_market_days(x_y_sorted.index.levels[0][-1], -ahead_days)]
    x_y_val = x_y_sorted.loc[x_y_sorted.index.levels[0][-1]:]
    
    x_train = x_y_train.iloc[:,:-1]
    x_val = x_y_val.iloc[:,:-1]
    y_train_true = x_y_train.iloc[:,-1]
    y_val_true = x_y_val.iloc[:,-1]
    
    estimator.fit(x_train)
    y_train_pred = estimator.predict(x_train)
    y_val_pred = estimator.predict(x_val)
    
    y_train_true_df = pd.DataFrame(y_train_true)
    y_train_pred_df = pd.DataFrame(y_train_pred)
    y_val_true_df = pd.DataFrame(y_val_true)
    y_val_pred_df = pd.DataFrame(y_val_pred)
    
    return y_train_true_df, y_train_pred_df, y_val_true_df, y_val_pred_df

In [17]:
from predictor import dummy_mean_predictor as dmp

predictor = dmp.DummyPredictor()

In [18]:
y_train_true, y_train_pred, y_val_true, y_val_pred = run_single_val(x, y, 1, predictor)
print(y_train_true.shape)
print(y_train_pred.shape)
print(y_val_true.shape)
print(y_val_pred.shape)

(314475, 1)
(314475, 1)
(479, 1)
(479, 1)


In [19]:
y_val_true

target
2014-12-16 A      47.50
           AAL     2.65
           AAP    56.28
           AAPL  167.44
           ABBV  113.99
           ABC    59.97
           ABT   160.00
           ACN    66.98
           ADBE   91.26
           ADI   149.12
           ADM    54.02
           ADP    74.56
           ADS    47.48
           ADSK   17.98
           AEE    25.19
           AEP    53.98
           AES    53.42
           AET    85.20
           AFL    61.03
           AGN    69.06
           AIG   163.46
           AIV    20.42
           AIZ   618.76
           AJG   139.66
           AKAM   14.22
           ALB    61.94
           ALK    89.84
           ALL    60.89
           ALLE   38.02
           ALXN   56.42
...                 ...
           VRSN   66.35
           VRTX   58.58
           VTR    50.29
           VZ     64.76
           WAT    48.40
           WBA   113.09
           WDC    45.63
           WEC   192.45
           WFC   119.89
           WFM    86.32
           WHR    36.63
           WLTW  110.88
           WM     55.28
           WMB   150.37
           WU     86.91
           WY     18.09
           WYN    36.58
           WYNN   43.92
           XEC    37.12
           XEL    93.21
           XLNX   14.14
           XOM    20.59
           XRAY   58.40
           XRX    53.84
           XYL    38.95
           YHOO   50.86
           YUM    73.14
           ZBH   114.17
           ZION   28.56
           ZTS    44.20

[479 rows x 1 columns]

In [20]:
pd.DataFrame(y_train_pred).reset_index()

,level_0,level_1,0
0,1993-01-29,AAL,19.730000
1,1993-01-29,ABBV,2.111429
2,1993-01-29,ABC,2.595714
3,1993-01-29,ADI,22.822857
4,1993-01-29,ADP,15.481429
5,1993-01-29,ADS,7.552857
6,1993-01-29,ADSK,13.665714
7,1993-01-29,AEE,1.165714
8,1993-01-29,AEP,2.401429
9,1993-01-29,AES,11.202857


In [21]:
grouped_df = pd.DataFrame(y_train_pred).reset_index().groupby('level_0').apply(lambda x: x.reset_index(drop=True)).drop('level_0', axis=1)
grouped_df.index = grouped_df.index.droplevel(level=1)
grouped_df.rename(columns={'level_1':'ticker'}, inplace=True)
grouped_df

,ticker,0
level_0,,
1993-01-29,AAL,19.730000
1993-01-29,ABBV,2.111429
1993-01-29,ABC,2.595714
1993-01-29,ADI,22.822857
1993-01-29,ADP,15.481429
1993-01-29,ADS,7.552857
1993-01-29,ADSK,13.665714
1993-01-29,AEE,1.165714
1993-01-29,AEP,2.401429


In [22]:
grouped_df.set_index('ticker', append=True).unstack()

0                                                \
ticker              A        AAL        AAP        AAPL        ABBV   
level_0                                                               
1993-01-29        NaN  19.730000        NaN         NaN    2.111429   
1993-02-09        NaN  20.447143        NaN         NaN    1.954286   
1993-02-19        NaN  21.767143        NaN         NaN    1.934286   
1993-03-02        NaN  23.177143        NaN         NaN    1.984286   
1993-03-11        NaN  23.410000        NaN         NaN    1.991429   
1993-03-22        NaN  22.607143        NaN         NaN    1.892857   
1993-03-31        NaN  23.250000        NaN         NaN    1.798571   
1993-04-12        NaN  25.607143        NaN         NaN    1.741429   
1993-04-21        NaN  29.784286        NaN         NaN    1.785714   
1993-04-30        NaN  29.285714        NaN         NaN    1.911429   
1993-05-11        NaN  28.714286        NaN         NaN    1.974286   
1993-05-20        NaN  29.232857        NaN         NaN    2.050000   
1993-06-01        NaN  26.142857        NaN         NaN    1.887143   
1993-06-10        NaN  21.267143        NaN         NaN    1.525714   
1993-06-21        NaN  21.804286        NaN         NaN    1.441429   
1993-06-30        NaN  24.161429        NaN         NaN    1.344286   
1993-07-12        NaN  24.195714        NaN         NaN    1.165714   
1993-07-21        NaN  24.395714        NaN         NaN    0.952857   
1993-07-30        NaN  26.178571        NaN         NaN    1.040000   
1993-08-10        NaN  30.411429        NaN         NaN    0.991429   
1993-08-19        NaN  30.910000        NaN         NaN    0.981429   
1993-08-30        NaN  30.071429        NaN         NaN    0.935714   
1993-09-09        NaN  28.322857        NaN         NaN    0.900000   
1993-09-20        NaN  29.715714        NaN         NaN    0.887143   
1993-09-29        NaN  24.411429        NaN         NaN    0.828571   
1993-10-08        NaN  20.195714        NaN         NaN    0.892857   
1993-10-19        NaN  19.610000        NaN         NaN    1.057143   
1993-10-28        NaN  19.697143        NaN         NaN    1.131429   
1993-11-08        NaN  19.552857        NaN         NaN    1.125714   
1993-11-17        NaN  18.660000        NaN         NaN    1.181429   
...               ...        ...        ...         ...         ...   
2014-02-18  39.811429   3.700000  51.131429  128.897143   75.648571   
2014-02-27  40.371429   3.740000  51.181429  128.662857   75.641429   
2014-03-10  40.530000   3.852857  51.101429  129.200000   75.860000   
2014-03-19  40.418571   4.018571  52.898571  129.768571   76.617143   
2014-03-28  40.582857   3.987143  53.448571  131.940000   76.568571   
2014-04-08  40.800000   3.818571  52.947143  133.602857   74.601429   
2014-04-17  41.041429   4.092857  52.635714  134.560000   78.497143   
2014-04-29  40.857143   4.072857  51.061429  137.342857   84.780000   
2014-05-08  39.290000   3.965714  51.385714  139.785714   84.497143   
2014-05-19  38.640000   4.044286  52.127143  140.192857   87.482857   
2014-05-29  39.252857   4.017143  52.461429  142.290000   91.272857   
2014-06-09  38.535714   4.291429  55.570000  139.524286   92.808571   
2014-06-18  39.918571   4.114286  54.348571  141.692857   91.045714   
2014-06-27  40.117143   4.225714  54.347143  142.522857   93.894286   
2014-07-09  39.710000   4.470000  53.945714  145.041429   95.041429   
2014-07-18  40.117143   3.770000  51.997143  147.490000   96.285714   
2014-07-29  38.398571   3.955714  50.058571  149.414286   96.275714   
2014-08-07  38.171429   4.108571  50.147143  150.678571   96.271429   
2014-08-18  39.367143   4.231429  52.057143  153.967143  100.655714   
2014-08-27  39.724286   4.160000  50.704286  154.108571  100.887143   
2014-09-08  38.805714   4.007143  49.494286  149.344286  100.418571   
2014-09-17  38.327143   3.755714  49.877143  143.851429  101.092857   
2014-09-26  38.504286   3.437143  48.584286 

In [23]:
compare_ready_df = grouped_df.set_index('ticker', append=True).unstack()
compare_ready_df.columns = compare_ready_df.columns.droplevel(level=0)
compare_ready_df.index.name = 'date'
compare_ready_df

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,19.730000,NaN,NaN,2.111429,2.595714,NaN,NaN,NaN,22.822857,...,14.224286,10.175714,NaN,7.888571,NaN,NaN,NaN,NaN,11.374286,NaN
1993-02-09,NaN,20.447143,NaN,NaN,1.954286,2.451429,NaN,NaN,NaN,22.285714,...,14.102857,10.068571,NaN,7.261429,NaN,NaN,NaN,NaN,11.155714,NaN
1993-02-19,NaN,21.767143,NaN,NaN,1.934286,2.402857,NaN,NaN,NaN,21.671429,...,13.700000,10.248571,NaN,6.497143,NaN,NaN,NaN,NaN,11.205714,NaN
1993-03-02,NaN,23.177143,NaN,NaN,1.984286,2.418571,NaN,NaN,NaN,22.345714,...,13.998571,10.571429,NaN,6.847143,NaN,NaN,NaN,NaN,11.282857,NaN
1993-03-11,NaN,23.410000,NaN,NaN,1.991429,2.430000,NaN,NaN,NaN,22.285714,...,13.835714,10.688571,NaN,7.540000,NaN,NaN,NaN,NaN,11.615714,NaN
1993-03-22,NaN,22.607143,NaN,NaN,1.892857,2.398571,NaN,NaN,NaN,22.221429,...,13.804286,10.842857,NaN,7.067143,NaN,NaN,NaN,NaN,12.142857,NaN
1993-03-31,NaN,23.250000,NaN,NaN,1.798571,2.071429,NaN,NaN,NaN,21.311429,...,13.500000,11.067143,NaN,7.010000,NaN,NaN,NaN,NaN,12.070000,NaN
1993-04-12,NaN,25.607143,NaN,NaN,1.741429,1.971429,NaN,NaN,NaN,21.500000,...,13.337143,11.362857,NaN,6.032857,NaN,NaN,NaN,NaN,11.794286,NaN
1993-04-21,NaN,29.784286,NaN,NaN,1.785714,1.862857,NaN,NaN,NaN,21.721429,...,12.881429,10.978571,NaN,6.230000,NaN,NaN,NaN,NaN,10.938571,NaN


In [24]:
reshape_by_symbol(y_train_pred)

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,19.730000,NaN,NaN,2.111429,2.595714,NaN,NaN,NaN,22.822857,...,14.224286,10.175714,NaN,7.888571,NaN,NaN,NaN,NaN,11.374286,NaN
1993-02-09,NaN,20.447143,NaN,NaN,1.954286,2.451429,NaN,NaN,NaN,22.285714,...,14.102857,10.068571,NaN,7.261429,NaN,NaN,NaN,NaN,11.155714,NaN
1993-02-19,NaN,21.767143,NaN,NaN,1.934286,2.402857,NaN,NaN,NaN,21.671429,...,13.700000,10.248571,NaN,6.497143,NaN,NaN,NaN,NaN,11.205714,NaN
1993-03-02,NaN,23.177143,NaN,NaN,1.984286,2.418571,NaN,NaN,NaN,22.345714,...,13.998571,10.571429,NaN,6.847143,NaN,NaN,NaN,NaN,11.282857,NaN
1993-03-11,NaN,23.410000,NaN,NaN,1.991429,2.430000,NaN,NaN,NaN,22.285714,...,13.835714,10.688571,NaN,7.540000,NaN,NaN,NaN,NaN,11.615714,NaN
1993-03-22,NaN,22.607143,NaN,NaN,1.892857,2.398571,NaN,NaN,NaN,22.221429,...,13.804286,10.842857,NaN,7.067143,NaN,NaN,NaN,NaN,12.142857,NaN
1993-03-31,NaN,23.250000,NaN,NaN,1.798571,2.071429,NaN,NaN,NaN,21.311429,...,13.500000,11.067143,NaN,7.010000,NaN,NaN,NaN,NaN,12.070000,NaN
1993-04-12,NaN,25.607143,NaN,NaN,1.741429,1.971429,NaN,NaN,NaN,21.500000,...,13.337143,11.362857,NaN,6.032857,NaN,NaN,NaN,NaN,11.794286,NaN
1993-04-21,NaN,29.784286,NaN,NaN,1.785714,1.862857,NaN,NaN,NaN,21.721429,...,12.881429,10.978571,NaN,6.230000,NaN,NaN,NaN,NaN,10.938571,NaN


In [25]:
reshape_by_symbol(y_train_true)

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,19.75,NaN,NaN,2.03,2.44,NaN,NaN,NaN,22.75,...,14.40,10.29,NaN,7.25,NaN,NaN,NaN,NaN,11.56,NaN
1993-02-09,NaN,22.12,NaN,NaN,1.96,2.50,NaN,NaN,NaN,21.38,...,13.82,10.04,NaN,6.75,NaN,NaN,NaN,NaN,10.69,NaN
1993-02-19,NaN,22.25,NaN,NaN,1.94,2.44,NaN,NaN,NaN,21.62,...,13.97,10.49,NaN,6.42,NaN,NaN,NaN,NaN,11.50,NaN
1993-03-02,NaN,23.75,NaN,NaN,2.03,2.44,NaN,NaN,NaN,22.69,...,14.05,10.74,NaN,7.50,NaN,NaN,NaN,NaN,11.75,NaN
1993-03-11,NaN,22.25,NaN,NaN,1.90,2.39,NaN,NaN,NaN,22.00,...,13.59,10.91,NaN,6.80,NaN,NaN,NaN,NaN,12.00,NaN
1993-03-22,NaN,23.00,NaN,NaN,1.84,2.38,NaN,NaN,NaN,22.06,...,13.88,10.84,NaN,7.42,NaN,NaN,NaN,NaN,12.00,NaN
1993-03-31,NaN,25.88,NaN,NaN,1.79,2.06,NaN,NaN,NaN,21.19,...,13.32,11.38,NaN,6.34,NaN,NaN,NaN,NaN,12.06,NaN
1993-04-12,NaN,29.00,NaN,NaN,1.77,1.94,NaN,NaN,NaN,21.31,...,13.20,11.18,NaN,6.19,NaN,NaN,NaN,NaN,11.44,NaN
1993-04-21,NaN,28.12,NaN,NaN,1.83,1.88,NaN,NaN,NaN,21.88,...,12.84,10.94,NaN,6.04,NaN,NaN,NaN,NaN,10.56,NaN


In [26]:
reshape_by_symbol(y_val_pred)

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-12-16,44.962857,2.592857,56.491429,165.978571,111.152857,58.154286,159.715714,66.425714,88.905714,144.894286,...,13.825714,20.055714,57.662857,53.691429,38.012857,50.368571,71.494286,113.312857,28.001429,42.694286


In [27]:
reshape_by_symbol(y_val_true)

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-12-16,47.5,2.65,56.28,167.44,113.99,59.97,160.0,66.98,91.26,149.12,...,14.14,20.59,58.4,53.84,38.95,50.86,73.14,114.17,28.56,44.2


In [28]:
r2_score(y_val_true, y_val_pred) # Of course this is not what we want to calculate for evaluation or validation...

0.99867098911463936

In [29]:
pp.fill_missing(reshape_by_symbol(y_train_pred)).head()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,42.81,19.730000,39.49,20.924,2.111429,2.595714,14.08,34.37,15.01,22.822857,...,14.224286,10.175714,6.907143,7.888571,24.916667,1.306667,7.474,28.781429,11.374286,31.023333
1993-02-09,42.81,20.447143,39.49,20.924,1.954286,2.451429,14.08,34.37,15.01,22.285714,...,14.102857,10.068571,6.907143,7.261429,24.916667,1.306667,7.474,28.781429,11.155714,31.023333
1993-02-19,42.81,21.767143,39.49,20.924,1.934286,2.402857,14.08,34.37,15.01,21.671429,...,13.700000,10.248571,6.907143,6.497143,24.916667,1.306667,7.474,28.781429,11.205714,31.023333
1993-03-02,42.81,23.177143,39.49,20.924,1.984286,2.418571,14.08,34.37,15.01,22.345714,...,13.998571,10.571429,6.907143,6.847143,24.916667,1.306667,7.474,28.781429,11.282857,31.023333
1993-03-11,42.81,23.410000,39.49,20.924,1.991429,2.430000,14.08,34.37,15.01,22.285714,...,13.835714,10.688571,6.907143,7.540000,24.916667,1.306667,7.474,28.781429,11.615714,31.023333


In [30]:
filled_df = pp.fill_missing(reshape_by_symbol(y_train_pred.dropna(how='all', axis=0)))
filled_df.isnull().sum().sum()

0

In [31]:
filled_df.head()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,42.81,19.730000,39.49,20.924,2.111429,2.595714,14.08,34.37,15.01,22.822857,...,14.224286,10.175714,6.907143,7.888571,24.916667,1.306667,7.474,28.781429,11.374286,31.023333
1993-02-09,42.81,20.447143,39.49,20.924,1.954286,2.451429,14.08,34.37,15.01,22.285714,...,14.102857,10.068571,6.907143,7.261429,24.916667,1.306667,7.474,28.781429,11.155714,31.023333
1993-02-19,42.81,21.767143,39.49,20.924,1.934286,2.402857,14.08,34.37,15.01,21.671429,...,13.700000,10.248571,6.907143,6.497143,24.916667,1.306667,7.474,28.781429,11.205714,31.023333
1993-03-02,42.81,23.177143,39.49,20.924,1.984286,2.418571,14.08,34.37,15.01,22.345714,...,13.998571,10.571429,6.907143,6.847143,24.916667,1.306667,7.474,28.781429,11.282857,31.023333
1993-03-11,42.81,23.410000,39.49,20.924,1.991429,2.430000,14.08,34.37,15.01,22.285714,...,13.835714,10.688571,6.907143,7.540000,24.916667,1.306667,7.474,28.781429,11.615714,31.023333


### That looks like the best way to represent the data if we want to calculate the $R^2$ distance on a per-symbol basis. I could add it to the single val function.

In [58]:
def run_single_val(x, y, ahead_days, estimator):
    multiindex = x.index.nlevels > 1
    
    x_y = pd.concat([x, y], axis=1)
    x_y_sorted = x_y.sort_index()
    if multiindex:
        x_y_train = x_y_sorted.loc[:fe.add_market_days(x_y_sorted.index.levels[0][-1], -ahead_days)]
        x_y_val = x_y_sorted.loc[x_y_sorted.index.levels[0][-1]:]
    else:
        x_y_train = x_y_sorted.loc[:fe.add_market_days(x_y_sorted.index[-1], -ahead_days)]
        x_y_val = x_y_sorted.loc[x_y_sorted.index[-1]:]
    
    x_train = x_y_train.iloc[:,:-1]
    x_val = x_y_val.iloc[:,:-1]
    y_train_true = x_y_train.iloc[:,-1]
    y_val_true = x_y_val.iloc[:,-1]
    
    estimator.fit(x_train)
    y_train_pred = estimator.predict(x_train)
    y_val_pred = estimator.predict(x_val)
    
    y_train_true_df = pd.DataFrame(y_train_true)
    y_train_pred_df = pd.DataFrame(y_train_pred)
    y_val_true_df = pd.DataFrame(y_val_true)
    y_val_pred_df = pd.DataFrame(y_val_pred)
    
    return y_train_true, \
           y_train_pred, \
           y_val_true, \
           y_val_pred

In [59]:
y_train_true, y_train_pred, y_val_true, y_val_pred = run_single_val(x, y, 1, predictor)
print(y_train_true.shape)
print(y_train_pred.shape)
print(y_val_true.shape)
print(y_val_pred.shape)

(380121,)
(380121,)
(483,)
(483,)


In [60]:
print(y_train_true.shape)
y_train_true.head()

(380121,)


1993-01-29  A       42.81
            AAL     19.75
            AAP     39.90
            AAPL    20.88
            ABBV     2.03
Name: target, dtype: float64

In [61]:
y = y_train_true
multiindex = y.index.nlevels > 1
    
if multiindex:
    DATE_LEVEL_NAME = 'level_0'
else:
    DATE_LEVEL_NAME = 'index'

In [62]:
DATE_LEVEL_NAME

'level_0'

In [63]:
y.reset_index()

,level_0,level_1,target
0,1993-01-29,A,42.81
1,1993-01-29,AAL,19.75
2,1993-01-29,AAP,39.90
3,1993-01-29,AAPL,20.88
4,1993-01-29,ABBV,2.03
5,1993-01-29,ABC,2.44
6,1993-01-29,ABT,13.91
7,1993-01-29,ACN,35.00
8,1993-01-29,ADBE,15.17
9,1993-01-29,ADI,22.75


In [64]:
reshape_by_symbol(y_train_true)

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,42.81,19.75,39.90,20.88,2.03,2.44,13.91,35.00,15.17,22.75,...,14.40,10.29,6.50,7.25,24.25,1.38,7.28,29.76,11.56,31.01
1993-02-09,42.81,22.12,39.90,20.88,1.96,2.50,13.91,35.00,15.17,21.38,...,13.82,10.04,6.50,6.75,24.25,1.38,7.28,29.76,10.69,31.01
1993-02-19,42.81,22.25,39.90,20.88,1.94,2.44,13.91,35.00,15.17,21.62,...,13.97,10.49,6.50,6.42,24.25,1.38,7.28,29.76,11.50,31.01
1993-03-02,42.81,23.75,39.90,20.88,2.03,2.44,13.91,35.00,15.17,22.69,...,14.05,10.74,6.50,7.50,24.25,1.38,7.28,29.76,11.75,31.01
1993-03-11,42.81,22.25,39.90,20.88,1.90,2.39,13.91,35.00,15.17,22.00,...,13.59,10.91,6.50,6.80,24.25,1.38,7.28,29.76,12.00,31.01
1993-03-22,42.81,23.00,39.90,20.88,1.84,2.38,13.91,35.00,15.17,22.06,...,13.88,10.84,6.50,7.42,24.25,1.38,7.28,29.76,12.00,31.01
1993-03-31,42.81,25.88,39.90,20.88,1.79,2.06,13.91,35.00,15.17,21.19,...,13.32,11.38,6.50,6.34,24.25,1.38,7.28,29.76,12.06,31.01
1993-04-12,42.81,29.00,39.90,20.88,1.77,1.94,13.91,35.00,15.17,21.31,...,13.20,11.18,6.50,6.19,24.25,1.38,7.28,29.76,11.44,31.01
1993-04-21,42.81,28.12,39.90,20.88,1.83,1.88,13.91,35.00,15.17,21.88,...,12.84,10.94,6.50,6.04,24.25,1.38,7.28,29.76,10.56,31.01


### Now, let's implement the rolling validation.

In [65]:
train_eval_days = -1 # In market days
base_days = 7 # In market days
step_days = 7 # market days
ahead_days = 1 # market days
today = data_df.index[-1] # Real date

train_days = 252 # market days per training period
step_eval_days =  30 # market days between training periods beginings

filled_data_df = pp.fill_missing(data_df)

tic = time()
x, y = fe.generate_train_intervals(filled_data_df, 
                                   train_eval_days, 
                                   base_days, 
                                   step_days, 
                                   ahead_days, 
                                   today, 
                                   fe.feature_close_one_to_one)
toc = time()
print('Elapsed time: %i seconds.' % (toc-tic))

Elapsed time: 143 seconds.


In [66]:
x_y_sorted = pd.concat([x, y], axis=1).sort_index()
x_y_sorted

0       1       2       3       4       5       6  \
1993-01-29 A      42.81   42.81   42.81   42.81   42.81   42.81   42.81   
           AAL    18.75   19.12   20.25   20.50   20.12   19.62   19.75   
           AAP    39.90   39.90   39.90   39.90   39.90   39.90   39.90   
           AAPL   20.88   20.88   20.88   20.88   20.88   20.88   20.88   
           ABBV    2.12    2.19    2.15    2.14    2.12    2.04    2.02   
           ABC     2.62    2.62    2.56    2.62    2.69    2.62    2.44   
           ABT    13.91   13.91   13.91   13.91   13.91   13.91   13.91   
           ACN    35.00   35.00   35.00   35.00   35.00   35.00   35.00   
           ADBE   15.17   15.17   15.17   15.17   15.17   15.17   15.17   
           ADI    21.94   22.38   22.31   22.69   23.19   24.00   23.25   
           ADM    20.60   20.60   20.60   20.60   20.60   20.60   20.60   
           ADP    15.25   15.25   15.06   15.56   15.69   15.81   15.75   
           ADS     7.00    7.06    7.25    7.25    8.00    8.12    8.19   
           ADSK   13.44   13.50   13.66   13.72   14.00   13.75   13.59   
           AEE     1.09    1.13    1.19    1.20    1.21    1.17    1.17   
           AEP     2.41    2.41    2.41    2.41    2.41    2.39    2.37   
           AES    11.24   11.03   11.35   11.24   11.24   11.24   11.08   
           AET    11.53   11.79   11.87   12.21   12.35   12.46   12.49   
           AFL     5.94    5.97    5.89    5.94    5.80    5.45    5.66   
           AGN    24.70   24.70   24.70   24.70   24.70   24.70   24.70   
           AIG     7.75    7.50    7.38    7.16    7.47    6.73    6.22   
           AIV     0.94    0.94    0.94    0.94    0.94    0.94    0.94   
           AIZ    17.56   17.12   17.19   17.88   18.12   18.06   17.62   
           AJG    10.33   10.33   10.33   10.33   10.33   10.33   10.33   
           AKAM    4.41    4.53    4.53    4.49    4.49    4.37    4.45   
           ALB     4.49    4.52    4.57    4.65    4.84    4.80    4.82   
           ALK     6.42    6.64    6.62    6.50    6.73    6.66    6.67   
           ALL     7.25    7.25    7.25    7.25    7.25    7.25    7.25   
           ALLE   18.50   18.50   18.50   18.50   18.50   18.50   18.50   
           ALXN  250.30  252.42  257.96  265.88  268.26  264.83  261.92   
...                 ...     ...     ...     ...     ...     ...     ...   
2014-12-16 VRTX   55.54   56.26   57.30   58.29   58.49   58.46   58.28   
           VTR    47.03   47.79   47.07   48.88   48.84   49.36   49.79   
           VZ     62.77   63.75   64.85   64.70   65.15   64.92   64.88   
           WAT    47.88   48.56   48.96   48.80   49.07   49.07   48.67   
           WBA   104.49  107.95  112.36  113.88  114.28  113.30  113.11   
           WDC    41.84   43.63   44.80   46.00   45.64   45.73   45.35   
           WEC   174.43  188.93  190.52  189.73  190.29  191.13  191.15   
           WFC   114.99  118.22  118.38  119.20  119.28  120.05  119.95   
           WFM    81.48   82.90   84.31   84.76   84.71   85.16   85.68   
           WHR    34.74   35.01   35.30   36.19   36.49   36.58   36.62   
           WLTW  100.94  106.82  108.67  113.62  111.77  113.62  111.95   
           WM     52.77   53.79   55.21   54.45   54.97   55.47   55.34   
           WMB   140.52  138.97  145.10  150.11  147.48  151.69  150.90   
           WMT    76.20   76.20   76.20   76.20   76.20   76.20   76.20   
           WU     82.96   84.23   85.94   85.16   86.38   86.66   86.43   
           WY     16.95   17.47   17.78   18.00   18.05   18.12   18.07   
           WYN    34.22   34.75   35.35   35.40   35.65   35.43   36.23   
           WYNN   43.70   43.42   43.71   43.00   43.87   43.91   44.08   
           XEC    34.97   35.75   36.43   36.49   37.00   37.20   36.97   
           XEL    86.41   89.02   91.16   93.64   93.33   94.59   93.78   
           XLNX   13.35   13.46   13.89   13.96   13.97   14.07   14.08   
           XOM    19.31   19.60   20.09   20.07

In [67]:
start_date = x_y_sorted.index.levels[0][0]
start_date

Timestamp('1993-01-29 00:00:00')

In [68]:
end_date = fe.add_market_days(start_date, 252)
end_date

Timestamp('1994-01-27 00:00:00')

In [69]:
end_date = fe.add_index_days(start_date, 252, x_y_sorted)
end_date

Timestamp('2000-01-25 00:00:00')

### So, I could use a training period based in an amount of market days, or in an amount of sample base periods. The first approach would be taking into consideration the temporal correlation of the data, the second would consider that the amount of samples should be large enough. Not to lose sight of the real problem at hand, I will use the market days approach, and then check that the amount of samples is big enough.

In [70]:
end_date = fe.add_market_days(start_date, 252)
x_i = x_y_sorted.loc[start_date:end_date].iloc[:,:-1]
y_i = x_y_sorted.loc[start_date:end_date].iloc[:,-1]
print(x_i.shape)
print(x_i.head())
print(y_i.shape)
print(y_i.head())

(17871, 7)
                     0      1      2      3      4      5      6
1993-01-29 A     42.81  42.81  42.81  42.81  42.81  42.81  42.81
           AAL   18.75  19.12  20.25  20.50  20.12  19.62  19.75
           AAP   39.90  39.90  39.90  39.90  39.90  39.90  39.90
           AAPL  20.88  20.88  20.88  20.88  20.88  20.88  20.88
           ABBV   2.12   2.19   2.15   2.14   2.12   2.04   2.02
(17871,)
1993-01-29  A       42.81
            AAL     19.75
            AAP     39.90
            AAPL    20.88
            ABBV     2.03
Name: target, dtype: float64


In [74]:
ahead_days

1

In [71]:
predictor = dmp.DummyPredictor()

y_train_true, y_train_pred, y_val_true, y_val_pred = run_single_val(x_i, y_i, ahead_days, predictor)
print(y_train_true.shape)
print(y_train_pred.shape)
print(y_val_true.shape)
print(y_val_pred.shape)

(17871,)
(17871,)
(0,)
(0,)


In [72]:
y_train_pred.head()

1993-01-29  A       42.810000
            AAL     19.730000
            AAP     39.900000
            AAPL    20.880000
            ABBV     2.111429
dtype: float64

In [73]:
y_train_pred.dropna(axis=1, how='all').shape

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [ ]:
scores = r2_score(pp.fill_missing(y_train_pred), pp.fill_missing(y_train_true), multioutput='raw_values')
print('R^2 score = %f +/- %f' % (np.mean(scores), 2*np.std(scores)))

In [ ]:
scores = r2_score(y_train_pred, y_train_true, multioutput='raw_values')
print('R^2 score = %f +/- %f' % (np.mean(scores), np.std(scores)))

In [ ]:
len(scores)

In [ ]:
y_val_true_df = pd.DataFrame()

In [ ]:
y_val_true

In [ ]:
y_val_true_df.append(y_val_true)

### A lot of attention should be paid to the effect of filling the missing data. It may change the whole results.

In [ ]:
x.index.min()

In [ ]:
x.index.max()

In [ ]:
x.index.max() - x.index.min()

In [ ]:
(x.index.max() - fe.add_market_days(x.index.min(), train_days)).days // step_days

### That last number is an approximation of the number of train/evaluation sets that are being considered.

In [ ]:
def roll_evaluate(x, y, train_days, step_eval_days, ahead_days, verbose=False):
    """Warning: The final date of the period should be no larger than the final date of the SPY_DF"""
    
    # calculate start and end date
    # sort by date
    x_y_sorted = pd.concat([x, y], axis=1).sort_index()
    start_date = x_y_sorted.index[0]
    end_date = fe.add_market_days(start_date, train_days)
    final_date = x_y_sorted.index[-1]
    
    # loop: run_single_val(x,y, ahead_days, estimator)
    r2_train_means = []
    r2_train_stds = []
    y_val_true_df = pd.DataFrame()
    y_val_pred_df = pd.DataFrame()
    num_training_sets = (252/365) * (x.index.max() - fe.add_market_days(x.index.min(), train_days)).days // step_eval_days
    set_index = 0
    if verbose:
        print('Evaluating approximately %i training/evaluation pairs' % num_training_sets)
    
    while end_date < final_date:
        x = x_y_sorted.loc[start_date:end_date].iloc[:,:-1]
        y = x_y_sorted.loc[start_date:end_date].iloc[:,-1]
        y_train_true, y_train_pred, y_val_true, y_val_pred = run_single_val(x, y, ahead_days, predictor)
        # Calculate R^2 for training and append
        scores = r2_score(y_train_true, y_train_pred, multioutput='raw_values')
        r2_train_means.append(np.mean(scores))
        r2_train_stds.append(np.std(scores))
        # Append validation results
        y_val_true_df = y_val_true_df.append(y_val_true)
        y_val_pred_df = y_val_pred_df.append(y_val_pred)
        
        # Update the dates
        start_date = fe.add_market_days(start_date, step_eval_days)
        end_date = fe.add_market_days(end_date, step_eval_days)
        
        set_index += 1
        if verbose:
            sys.stdout.write('\rApproximately %2.1f percent complete.    ' % (100.0 * set_index / num_training_sets))
            sys.stdout.flush()
    
    return r2_train_means, r2_train_stds, y_val_true_df, y_val_pred_df

### Let's test the whole process

In [ ]:
train_eval_days = -1 # In market days
base_days = 14 # In market days
step_days = 30 # market days
ahead_days = 1 # market days
today = data_df.index[-1] # Real date

filled_data_df = pp.fill_missing(data_df)

tic = time()
x, y = fe.generate_train_intervals(filled_data_df, 
                                   train_eval_days, 
                                   base_days, 
                                   step_days, 
                                   ahead_days, 
                                   today, 
                                   fe.feature_close_one_to_one)
toc = time()
print('Elapsed time: %i seconds.' % (toc-tic))

In [ ]:
train_days = 252 # market days per training period
step_eval_days =  10 # market days between training periods beginings

tic = time()
r2_train_means, r2_train_stds, y_val_true_df, y_val_pred_df = roll_evaluate(x, y, train_days, step_eval_days, ahead_days, verbose=True)
toc = time()
print('Elapsed time: %i seconds.' % (toc-tic))

In [ ]:
y_val_true_df.head()

In [ ]:
pd.DataFrame(r2_train_means).describe()

In [ ]:
scores = r2_score(y_val_true_df.T, y_val_pred_df.T, multioutput='raw_values')
print('R^2 score = %f +/- %f' % (np.mean(scores), np.std(scores)))

In [ ]:
pd.DataFrame(scores).describe()

In [ ]:
plt.plot(y_val_true_df.index, r2_train_means, label='r2_train_means')
plt.plot(y_val_true_df.index, scores, label='r2 validation scores')
plt.legend(loc='lower left')

In [ ]:
scores_val = r2_score(y_val_true_df, y_val_pred_df, multioutput='raw_values')
print('R^2 score = %f +/- %f' % (np.mean(scores_val), np.std(scores_val)))

In [ ]:
plt.plot(scores_val, label='r2 validation scores')

In [ ]:
sorted_means = x.sort_index().mean(axis=1)

In [ ]:
sorted_means.head()

In [ ]:
sorted_means.plot()

In [ ]:
sub_period = sorted_means['2009-03-01':]
plt.scatter(sub_period.index, sub_period)

### It seems like the weird point, in which the model is predicting terribly may be the 2008 financial crisis. And the big unpredictability is limited to one symbol. I should implement a way to trace the symbols...

## What about the mean absolute error?

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
scores = mean_absolute_error(y_val_true_df.T, y_val_pred_df.T, multioutput='raw_values')
print('MAE score = %f +/- %f' % (np.mean(scores), np.std(scores)))

In [ ]:
plt.plot(y_val_true_df.index, scores, label='MAE validation scores')
plt.legend(loc='lower left')

In [ ]:
pd.DataFrame(scores).describe()

In [ ]:
scores = mean_absolute_error(y_val_true_df, y_val_pred_df, multioutput='raw_values')
print('MAE score = %f +/- %f' % (np.mean(scores), np.std(scores)))

In [ ]:
plt.plot(scores, label='MAE validation scores')
plt.legend(loc='lower left')

In [ ]:
pd.DataFrame(scores).describe()